In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt

In [7]:
from keras import models
from keras import layers
from keras import optimizers

image_size=50

# Create the model
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(image_size, image_size, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.25))
model.add(layers.Dense(8, activation='softmax')) 
 
# Check the trainable status of the individual layers
# for layer in model.layers:
#     print(layer, layer.trainable)

# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 50, 50, 128)       512       
_________________________________________________________________
flatten_3 (Flatten)          (None, 320000)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                20480064  
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 520       
Total params: 20,481,096
Trainable params: 20,481,096
Non-trainable params: 0
_________________________________________________________________


In [8]:
segment = 'color'
train_dir = 'output_color/{}/train'.format(segment)
validation_dir = 'output_color/{}/validation'.format(segment)
model_fname = 'fc_{}.h5'.format(segment)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=4,
    width_shift_range=5,
    height_shift_range=5,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
validation_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=validation_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 9289 images belonging to 8 classes.
Found 2323 images belonging to 8 classes.


In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Calculate class weight
from collections import Counter
counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save(model_fname)

Epoch 1/30
93/92 [==============================] - 78s 838ms/step - loss: 1.9173 - acc: 0.4019 - val_loss: 1.2129 - val_acc: 0.5428
Epoch 2/30
93/92 [==============================] - 78s 841ms/step - loss: 1.2978 - acc: 0.4839 - val_loss: 1.1377 - val_acc: 0.5704
Epoch 3/30
93/92 [==============================] - 77s 826ms/step - loss: 1.1870 - acc: 0.5494 - val_loss: 1.0949 - val_acc: 0.5936
Epoch 4/30
93/92 [==============================] - 72s 774ms/step - loss: 1.1232 - acc: 0.5849 - val_loss: 1.0387 - val_acc: 0.6272
Epoch 5/30
93/92 [==============================] - 71s 766ms/step - loss: 1.0657 - acc: 0.6074 - val_loss: 0.9784 - val_acc: 0.6595
Epoch 6/30
93/92 [==============================] - 73s 781ms/step - loss: 1.0260 - acc: 0.6309 - val_loss: 0.9520 - val_acc: 0.6651
Epoch 7/30
93/92 [==============================] - 73s 781ms/step - loss: 0.9828 - acc: 0.6512 - val_loss: 1.0366 - val_acc: 0.6156
Epoch 8/30
93/92 [==============================] - 77s 827ms/step - 

In [40]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

KeyError: 'acc'